# YFCC-100M

In [1]:
from pathlib import Path
data_dir = Path('/localdata/jsu068')

In [3]:
import numpy as np
from datasets import YFCC100MDataset

In [4]:
ds = YFCC100MDataset()
# ds.prepare()

In [6]:
ds.__str__()
# dir(ds)

'Dataset YFCC100MDataset in dimension 192, with distance euclidean, search_type knn_filtered, size: Q 100000 B 10000000'

In [7]:
ds.get_dataset_metadata().shape, ds.get_queries_metadata().shape

((10000000, 200386), (100000, 200386))

In [8]:
dataset_metadata = ds.get_dataset_metadata()  # (10000000, 200386)
dataset_metadata.data.shape

(108210476,)

In [9]:
query_metadata = ds.get_queries_metadata()
query_metadata.indices.shape

(138374,)

In [10]:
rows, cols = dataset_metadata.nonzero()
rows.shape, cols.shape

((108210476,), (108210476,))

In [11]:
len(set(cols))

200363

In [12]:
dataset_metadata.nonzero()

(array([      0,       0,       0, ..., 9999999, 9999999, 9999999],
       dtype=int32),
 array([    6,    15,    24, ...,   345, 48730,  2841], dtype=int32))

In [13]:
from collections import defaultdict

filter_dict = defaultdict(list)
for idx, v in zip(rows, cols):
    filter_dict[idx].append(v)

In [14]:
len(filter_dict)

9998928

In [15]:
# {k: v for k, v in filter_dict.items() if not v}
not_in_list = [i for i in range(10000000) if i not in filter_dict]
len(not_in_list)

1072

In [17]:
import os

labels_dir = data_dir / "yfcc"
file_path = os.path.join(labels_dir, "yfcc.filter.base.txt")
label_type = np.int32

data = []
it = 0

for idx, labels in filter_dict.items():
    while it < len(not_in_list) and idx > not_in_list[it]:
        data.append([0])
        it += 1
    data.append(labels)

assert(len(data) == 10000000)
total_labels = 0

cnt = 0
with open(file_path, "w") as f:
    for labels in data:
        cnt += 1
        label_written = False
        for label in labels:
            if label_written:
                f.write(',')
            f.write(str(label))
            label_written = True
            total_labels += 1
        if cnt < len(data):
            f.write('\n')

print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

10000000 points with 108211548 labels are written to /localdata/jsu068/yfcc/yfcc.filter.base.txt


In [14]:
# import os

# labels_dir = data_dir / "yfcc"
# file_path = os.path.join(labels_dir, "yfcc.filter.base.bin")
# label_type = np.int32

# data = []
# it = 0

# for idx, labels in filter_dict.items():
#     while it < len(not_in_list) and idx > not_in_list[it]:
#         data.append([0])
#         it += 1
#     data.append(labels)

# assert(len(data) == 10000000)
# total_labels = 0
# # separator = np.iinfo(label_type).max - 1
# separator = -1

# with open(file_path, "wb") as f:
#     # total points
#     shape = len(data)
#     f.write(shape.to_bytes(4, byteorder='little'))

#     # data
#     for labels in data:
#         label_array = np.array(labels, dtype=label_type)
#         # data
#         f.write(label_array.tobytes())
#         # separator
#         f.write(separator.to_bytes(4, byteorder='little', signed=True))
#         total_labels += label_array.shape[0]

# print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

10000000 points with 108211548 labels are written to /localdata/jsu068/yfcc/yfcc.filter.base.bin


In [18]:
single = []
for idx, labels in filter_dict.items():
    if len(labels) == 1:
        single.append(labels[0])
len(single), 0 in single

(13122, False)

In [19]:
rows, cols = query_metadata.nonzero()

In [20]:
filter_dict = defaultdict(list)
for row, col in zip(rows, cols):
    filter_dict[row].append(col)

In [21]:
len(filter_dict)

100000

In [23]:
import os

labels_dir = data_dir / "yfcc"
file_path = os.path.join(labels_dir, "yfcc.filter.query.txt")
label_type = np.int32

data = []
it = 0

for idx, labels in filter_dict.items():
    data.append(labels)

assert(len(data) == 100000)
total_labels = 0

cnt = 0
with open(file_path, "w") as f:
    for labels in data:
        cnt += 1
        label_written = False
        for label in labels:
            if label_written:
                f.write(',')
            f.write(str(label))
            label_written = True
            total_labels += 1
        if cnt < len(data):
            f.write('\n')

print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

100000 points with 138374 labels are written to /localdata/jsu068/yfcc/yfcc.filter.query.txt


In [ ]:
# import os

# file_path = os.path.join(labels_dir, "yfcc.filter.query.bin")

# data = []
# it = 0

# for idx, labels in filter_dict.items():
#     data.append(labels)

# assert(len(data) == 100000)
# total_labels = 0

# with open(file_path, "wb") as f:
#     # total points
#     shape = len(data)
#     f.write(shape.to_bytes(4, byteorder='little'))

#     # data
#     for labels in data:
#         label_array = np.array(labels, dtype=label_type)
#         # data
#         f.write(label_array.tobytes())
#         # separator
#         f.write(separator.to_bytes(4, byteorder='little', signed=True))
#         total_labels += label_array.shape[0]

# print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

100000 points with 138374 labels are written to /localdata/jsu068/yfcc/yfcc.filter.query.bin


# MS-MACRO

In [24]:
import numpy as np
import struct

In [25]:
from pathlib import Path
data_dir = Path('/localdata/jsu068')

macro_dir = data_dir / "marco"
embed_dir = macro_dir / "embedding"
query_dir = macro_dir / "query"

In [26]:
# embedding
with open(embed_dir / "marco.768D.100M.euclidean.fbin", "rb") as f:
    n = struct.unpack('I', f.read(4))[0]
    dim = struct.unpack('I', f.read(4))[0]

vectors = np.memmap(embed_dir / "marco.768D.100M.euclidean.fbin", dtype=np.float32, mode='r', shape=(n, dim))

n, dim, vectors.shape

(101070374, 768, (101070374, 768))

In [27]:
meta = np.fromfile(embed_dir / "meta.bin", dtype=np.uint8)
metaidx = np.fromfile(embed_dir / "metaidx.bin", dtype=np.int32)
meta.shape, metaidx.shape

((909486248,), (202140751,))

In [28]:
data = np.zeros(n, dtype=np.int32)
i = 0

for idx in list(metaidx)[2:]:
    if idx == 0:
        continue
    
    idx -= 1
    data[i] = meta[idx]
    i += 1
    
data.shape

(101070374,)

In [30]:
import os

file_path = os.path.join(embed_dir, "marco.filter.base.txt")
label_type = np.int32
total_labels = 0

cnt = 0
with open(file_path, "w") as f:
    # data
    for label in data:
        cnt += 1
        f.write(str(label))
        total_labels += 1
        if cnt < len(data):
            f.write('\n')

print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

101070374 points with 101070374 labels are written to /localdata/jsu068/marco/embedding/macro.filter.base.txt


In [29]:
# import os

# file_path = os.path.join(embed_dir, "marco.filter.base.bin")
# label_type = np.int32
# total_labels = 0
# # separator = np.iinfo(label_type).max
# separator = -1

# with open(file_path, "wb") as f:
#     # total points
#     shape = len(data)
#     f.write(shape.to_bytes(4, byteorder='little'))

#     # data
#     for label in data:
#         # label_array = np.array(labels, dtype=label_type)
#         # data
#         f.write(label.tobytes())
#         # separator
#         f.write(separator.to_bytes(4, byteorder='little', signed=True))
#         total_labels += 1

# print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

101070374 points with 101070374 labels are written to /localdata/jsu068/marco/embedding/macro.filter.base.bin


In [31]:
# query
with open(query_dir / "marco.768D.10K.euclidean.fbin", "rb") as f:
    n = struct.unpack('I', f.read(4))[0]
    dim = struct.unpack('I', f.read(4))[0]

vectors = np.memmap(query_dir / "marco.768D.10K.euclidean.fbin", dtype=np.float32, mode='r', shape=(n, dim))

n, dim, vectors.shape

(9376, 768, (9376, 768))

In [32]:
meta = np.fromfile(query_dir / "meta.bin", dtype=np.uint8)
metaidx = np.fromfile(query_dir / "metaidx.bin", dtype=np.int32)
meta.shape, metaidx.shape

((36397,), (18755,))

In [33]:
data = np.zeros(n, dtype=np.int32)
i = 0

for idx in list(metaidx)[2:]:
    if idx == 0:
        continue
    
    idx -= 1
    data[i] = meta[idx]
    i += 1
    
data.shape

(9376,)

In [35]:
import os

file_path = os.path.join(query_dir, "marco.filter.query.txt")
total_labels = 0

cnt = 0
with open(file_path, "w") as f:
    # data
    for label in data:
        cnt += 1
        f.write(str(label))
        total_labels += 1
        if cnt < len(data):
            f.write('\n')

print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

9376 points with 9376 labels are written to /localdata/jsu068/marco/query/marco.filter.query.txt


In [35]:
# import os

# file_path = os.path.join(query_dir, "marco.filter.query.bin")
# total_labels = 0

# with open(file_path, "wb") as f:
#     # total points
#     shape = len(data)
#     f.write(shape.to_bytes(4, byteorder='little'))

#     # data
#     for label in data:
#         # label_array = np.array(labels, dtype=label_type)
#         # data
#         f.write(label.tobytes())
#         # separator
#         f.write(separator.to_bytes(4, byteorder='little', signed=True))
#         total_labels += 1

# print(f"{len(data)} points with {total_labels} labels are written to {file_path}")

9376 points with 9376 labels are written to /localdata/jsu068/marco/query/macro.filter.query.bin
